# Note:
Due to Google Trends blocking scraping with a for loop, the list has been broken down into 3 items (or columns from a dataframe) at a time.

In [1]:
import pandas as pd
import datetime      
from pytrends.request import TrendReq
import time

In [2]:
df = pd.read_csv('google_trend_keyword.csv')
output_path = '../../../data/google_trends/'

In [3]:
df.head()

,general_terms,diagnosis,emotions,ngos,general terms,measures,impacts
0,mental illness,depression,unalive,Institute of Mental Health,covid-19,restriction,uncertainty
1,mental health,neurosis,help,IMH,epidemic,social distancing,death
2,self-harm,post traumatic stress disorder,dead,Samaritans Of Singapore,pandemic,circuit breaker,community cases
3,suicide,mood disorder,no motivation,SOS,pre-covid,quarantine,fatality rate
4,death,melancholia,loneliness,National Care Hotline,coronavirus,lockdown,cluster


In [12]:
# Highly recommended that you scrape 3 columns at a time as Google Trends will time you out
startTime = time.time()
pytrend = TrendReq(hl='en-US') # language of the query

category_list_1 = ['general_terms',
                 'diagnosis',
                 'emotions']

category_list_2 = ['ngos',
                   'general terms',
                   'measures']

category_list_3 = ['impacts']

def google_trends_scraper(category_list, df): # pass in the original df with the shorter list (max 3)
    filtered_df = df.loc[:,category_list]

    for idx, category in enumerate(category_list):
        colnames = category_list[idx]
        df_temp = filtered_df.iloc[:,idx].values.tolist()

        extracted_data = []

        for i in range(0,len(df_temp)):
            keywords = [df_temp[i]]
            pytrend.build_payload(
            kw_list=keywords,
            cat=0,
            timeframe='2019-01-01 2021-07-07',
            geo='SG')
            data = pytrend.interest_over_time()
            if not data.empty:
                data = data.drop(labels=['isPartial'],axis='columns')
                extracted_data.append(data)

        result = pd.concat(extracted_data, axis=1)
        result.to_csv(f'{output_path}{category}.csv')

        # executionTime = (time.time() - startTime)
        # print('Execution time in sec.: ' + str(executionTime))

# Load one cell at a time and wait for a couple seconds when the previous cell is done
Do not press all cells at a go. Google may deny your request

In [6]:
google_trends_scraper(category_list_1, df)

In [11]:
google_trends_scraper(category_list_2, df)

In [10]:
google_trends_scraper(category_list_3, df)